In [4]:
import requests
import pandas as pd

data = pd.read_csv("spam.csv", encoding = 'latin1')

# drop unavailable attributes
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
# rename columns 
data = data.rename(columns ={"v1":"target", "v2":"text"})
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, f1_score


def get_report(y_true, y_pred, model_name):
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, pos_label="spam")
    
    # Construct a single-row DataFrame with metrics
    report_df = pd.DataFrame({
        'Model': [model_name],
        'Accuracy': [accuracy],
        'F1 Score(+ve = spam)': [f1],
        'TP': [tp],
        'TN': [tn],
        'FP': [fp],
        'FN': [fn]
    })
    
    return report_df

In [7]:
request_json_arr = [{"text": text} for text in data["text"]]
result = data["target"]

In [20]:
url = "http://127.0.0.1:5000/predict"

responses = []
for request_json in request_json_arr:   
    response = requests.post(url, json=request_json)
    responses.append(response)

In [27]:
api_result = [response.json()["prediction"] for response in responses]
target = list(data["target"])

In [25]:
result = [1 if api_result[i] == target[i] else 0 for i in range(len(target))]
print("Correct Classifications: ", result.count(1))
print("InCorrect Classifications: ", result.count(0))

Correct Classifications:  5496
InCorrect Classifications:  76


In [38]:
get_report(data["target"], api_result, model_name="MNB_API")

,Model,Accuracy,F1 Score(+ve = spam),TP,TN,FP,FN
0,MNB_API,0.98636,0.949468,714,4782,43,33


In [ ]:
request_json = {"text_arr": list(data["text"])}
batch_url = "http://127.0.0.1:5000/batch_predict"
response = requests.post(batch_url, json=request_json)

In [ ]:
batch_api_result = [res for res in response.json()["predictions"]]
target = list(data["target"])

In [35]:
result = [1 if batch_api_result[i] == target[i] else 0 for i in range(len(target))]
print("Correct Classifications: ", result.count(1))
print("InCorrect Classifications: ", result.count(0))

Correct Classifications:  5496
InCorrect Classifications:  76


In [ ]:
get_report(data["target"], batch_api_result, model_name="MNB_BATCH_API")

,Model,Accuracy,F1 Score(+ve = spam),TP,TN,FP,FN
0,MNB_API,0.98636,0.949468,714,4782,43,33
